# Environmental Informatics

Modified from:

[ATM 623: Climate Modeling](../index.ipynb)

[Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany

Lecture 20: A peak at numerical methods for diffusion models

### About these notes:

This document uses the interactive [`Jupyter notebook`](https://jupyter.org) format. The notes can be accessed in several different ways:

- The interactive notebooks are hosted on `github` at https://github.com/brian-rose/ClimateModeling_courseware
- The latest versions can be viewed as static web pages [rendered on nbviewer](http://nbviewer.ipython.org/github/brian-rose/ClimateModeling_courseware/blob/master/index.ipynb)
- A complete snapshot of the notes as of May 2017 (end of spring semester) are [available on Brian's website](http://www.atmos.albany.edu/facstaff/brose/classes/ATM623_Spring2017/Notes/index.html).

[Also here is a legacy version from 2015](http://www.atmos.albany.edu/facstaff/brose/classes/ATM623_Spring2015/Notes/index.html).

Many of these notes make use of the `climlab` package, available at https://github.com/brian-rose/climlab

In [ ]:
#  Ensure compatibility with Python 2 and 3
from __future__ import print_function, division

## Contents

1. [The one-dimensional diffusion equation](#section1)
2. [Discretizing the diffusion operator in space](#section2)
3. [Coding the discretized diffusion operator in `numpy`](#section3)
4. [Discretizing the time derivative](#section4)
5. [Stability analysis of the FTCS scheme](#section5)
6. [Numerical tests with a shorter timestep](#section6)
7. [The need for a more efficient method](#section7)
8. [Implicit time method](#section8)
9. [Your homework assignment](#section9)

____________
<a id='section1'></a>

## 1. The one-dimensional diffusion equation
____________


Suppose that a concentration $C(x)$ is mixed down-gradient by a diffusive process.

The diffusive flux is

$$ F = - K \frac{\partial C}{\partial x} $$

There will be local changes in $C$ wherever this flux is convergent or divergent:

$$ \frac{\partial C}{\partial t} = - \frac{\partial F}{\partial x} $$

Putting this together gives the classical diffusion equation in one dimension

$$ \frac{\partial C}{\partial t} = \frac{\partial}{\partial x} \left( K \frac{\partial C}{\partial x} \right) $$

For simplicity, we are going to limit ourselves to Cartesian geometry rather than meridional diffusion on a sphere.

We will also assume here that $K$ is a constant, so our governing equation is

$$ \frac{\partial C}{\partial t} = K \frac{\partial^2 C}{\partial x^2} $$

This equation represents a time-dependent diffusion process. It is an **initial-boundary value problem**. We want to integrate the model forward in time to model the changes in the field $C(x)$.


## Challenge:
1. Find a particular solution to the equation above
2. If the flux is advected in the x direction, please write down the advection term.
3. If there is a source in the x direction, please add the source term to the equation.

Now, you have gotten an advection-diffusion-source equation

____________
<a id='section2'></a>

## 2. Discretizing the diffusion operator in space
____________



Solving a differential equation on a computer always requires some approximation to represent the continuous function $u(x,t)$ and its derivatives in terms of discrete quantities (arrays of numbers).

We have already dealt with simple discretization of the time derivative back in [Lecture 2](./Lecture02 -- Solving the zero-dimensional EBM.ipynb). We used the **forward Euler** method to step all our of radiation models forward in time so far.

### Some notation for discretization of $u(x,t)$

We will discretize time and space on grids

$$ x_j , ~~~ t^n $$

so that 

$$ C_j^n = C(x_j, ~t^n) $$

### Discretizing the diffusive flux

The governing equation can be written in terms of the convergence of the diffusive flux:

$$ \frac{\partial C}{\partial t} = - \frac{\partial F}{\partial x} $$

It is sensible to use a **centered difference** to approximate this derivative:

$$ \frac{\partial F}{\partial x} \bigg|_j \approx  \frac{F_{j+\frac{1}{2}} - F_{j-\frac{1}{2}}}{x_{j+\frac{1}{2}} - x_{j-\frac{1}{2}}} $$

The time tendency at point $x_j$ can thus be written

$$ \frac{\partial C}{\partial t} \bigg|_j  \approx - \frac{F_{j+\frac{1}{2}} - F_{j-\frac{1}{2}}}{x_{j+\frac{1}{2}} - x_{j-\frac{1}{2}}} $$

The flux itself depends on a spatial derivative of $u$. We will apply the same centered difference approximation. At point $x_j$ this would look like

$$ \frac{\partial C}{\partial x} \approx \frac{C_{j+\frac{1}{2}} - C_{j-\frac{1}{2}}}{x_{j+\frac{1}{2}} - x_{j-\frac{1}{2}}} $$

But we actually want to approximate $F_{j+\frac{1}{2}}$ and $F_{j-\frac{1}{2}}$, so we apply the centered difference formula at these intermediate points to get

$$ F_{j+\frac{1}{2}} \approx -K \frac{C_{j+1} - C_{j}}{x_{j+1} - x_{j}} $$

and

$$ F_{j-\frac{1}{2}} \approx -K \frac{C_{j} - C_{j-1}}{C_{j} - C_{j-1}} $$

Putting this all together, we can write the time tendency at $x_j$ as

$$ \frac{\partial C}{\partial t} \bigg|_j  \approx K \frac{ \frac{C_{j+1} - C_{j}}{x_{j+1} - x_{j}} - \frac{C_{j} - C_{j-1}}{x_{j} - x_{j-1}}}{x_{j+\frac{1}{2}} - x_{j-\frac{1}{2}}} $$

We'll make things easy on ourselves by using uniform grid spacing in $x$, so

$$ x_{j+1} - x_{j} = x_{j} - x_{j-1} = x_{j+\frac{1}{2}} - x_{j-\frac{1}{2}} = \Delta x $$

So our final formula for the diffusive flux convergence is

$$ \frac{\partial C}{\partial t} \bigg|_j  \approx K \frac{ C_{j+1} - 2 C_{j} + C_{j-1}}{\Delta x^2} $$

### No-flux boundary conditions

Suppose the domain is $0 \le x \le 1$, with solid walls at $x=0, 1$.

The physical boundary condition at the walls is that there can be no flux in or out of the walls:

$$ F(0) = F(1) = 0 $$

So the boundary conditions on $u$ are

$$ \frac{\partial C}{\partial x} = 0 ~~~ \text{at} ~~~ x=0,1 $$

### The staggered grid

Suppose we have a grid of $J+1$ total points between $x=0$ and $x=1$, **including the boundaries**:

- $x^*_0 = 0 $
- $x^*_1 = \Delta x$
- $x^*_2 = 2~\Delta x$
- ...
- $x^*_j = j~\Delta x$
- ...
- $x^*_{J-1} = (J-1)~\Delta x = 1 - \Delta x $
- $x^*_J = J ~ \Delta x = 1 $

Clearly then the grid spacing must be $\Delta x = 1/J$.

We'll define the fluxes on this grid. The boundary conditions can thus be written

$$ F_0 = F_J = 0 $$

Since our centered difference discretization defines $F$ at points halfway between the $u$ points, it is sensible to locate $u$ on another grid that is offset by $\Delta x / 2$.

The first grid point for $C$ is thus a distance $\Delta x / 2$ from the wall, and there are a total of $J$ points:

- $x_0 = \Delta x / 2$
- $x_1 = \Delta x / 2 + \Delta x$
- $x_2 = \Delta x / 2 + 2~\Delta x$
- ...
- $x_j = \Delta x / 2 + j~\Delta x$
- ...
- $x_{J-1} = \Delta x / 2 + (J-1)~\Delta x = 1 - \Delta x / 2 $

### Implementing the boundary condition on the staggered grid

At $x_0$ we have

$$ \frac{\partial C}{\partial t} \bigg|_0  \approx -\frac{ F_1 - F_0}{\Delta x} $$

Subbing in $F_0 = 0$ and the normal discretization for $F_1$ gives

$$ \frac{\partial C}{\partial t} \bigg|_0  \approx K \frac{ C_1 - C_0 }{\Delta x^2} $$

The same procedure at the other wall yields

$$ \frac{\partial C}{\partial t} \bigg|_{J-1}  \approx - K \frac{ C_{J-1} - C_{J-2} }{\Delta x^2} $$

Pulling this all together we have a complete discretization of the diffusion operator including its boundary conditions:

$$ \frac{\partial C}{\partial t} \bigg|_0  \approx K \frac{ C_1 - C_0 }{\Delta x^2} $$

$$ \frac{\partial C}{\partial t} \bigg|_j  \approx K \frac{ C_{j+1} - 2 C_{j} + C_{j-1}}{\Delta x^2}, ~~~~~~ j=1,...,J-2 $$

$$ \frac{\partial C}{\partial t} \bigg|_{J-1}  \approx - K \frac{ C_{J-1} - C_{J-2} }{\Delta x^2} $$



## Challenge:
Descretize the advection and source term in space

____________
<a id='section3'></a>

## 3. Coding the discretized diffusion operator in `numpy`
____________



In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex

Here we will divide our domain up into 20 grid points.

In [ ]:
J = 20
deltax = 1./J
display(Math(r'J = %i' %J))
display(Math(r'\Delta x = %0.3f' %deltax))

The fluxes will be solved on the staggered grid with 21 points.

$C$ will be solved on the 20 point grid.

In [ ]:
xstag = np.linspace(0., 1., J+1)
x = xstag[:-1] + deltax/2
print( x)

In [ ]:
C = np.zeros_like(x)

Here's one way to implement the finite difference, using array indexing.

In [ ]:
dCdx = (C[1:] - C[:-1]) / (x[1:] - x[:-1])

In [ ]:
dCdx.shape

We can also use the function `numpy.diff()` to accomplish the same thing: 

In [ ]:
help(np.diff)

In [ ]:
np.diff(C).shape

Here is a function that computes the diffusive flux $F$ on the staggered grid, including the boundaries.

In [ ]:
def diffusive_flux(C, deltax, K=1):
    #  Take the finite difference
    F = np.diff(C)/deltax
    #  add a zero as the first element (no flux on boundary)
    F = np.insert(F, 0, 0.)
    #  add another zero as the last element (no flux on boundary)
    F = np.append(F, 0.)
    #  flux is DOWN gradient, proportional to D
    return -K*F

In [ ]:
diffusive_flux(C,deltax).shape

The time tendency of $C$ is just the convergence of this flux, which requires one more finite difference:

In [ ]:
def diffusion(C, deltax, K=1):
    #  compute flux
    F = diffusive_flux(C, deltax, K)
    #  take convergence of flux
    return -np.diff(F) / deltax

### A smooth example

Suppose we have an initial $C$ field that has a local maximum in the interior.

The gaussian (bell curve) function is a convenient way to create such a field.

In [ ]:
def gaussian(x, mean, std):
    return np.exp(-(x-mean)**2/(2*std**2))/np.sqrt(2*np.pi*std**2)

In [ ]:
K = 0.01
C = gaussian(x, 0.5, 0.08)
dCdt = diffusion(C, deltax, K=K)
fig, ax = plt.subplots(1)
ax.plot(x, C, label='$C(x)$')
ax.plot(x, dCdt, label='$dC/dt$')
ax.legend()

Hopefully this makes sense. The diffusion is acting to smooth out $C$ by reducing the peak and increasing $u$ on the flanks of the gaussian bump.

### Some non-smooth examples

Use a random number generator to create some noisy initial conditions.

In [ ]:
fig = plt.figure(figsize=(10,8))
for n in range(4):
    C = np.random.random(J)
    dCdt = diffusion(C, deltax, K)
    ax = fig.add_subplot(2,2,n+1)
    ax.plot(x, C)
    ax.plot(x, dCdt)

____________
<a id='section4'></a>

## 4. Discretizing the time derivative
____________




The simplest way to discretize the time derivative is the **forward Euler** method:

$$ \frac{d C}{dt} \bigg|^n \approx \frac{C^{n+1} - C^n}{\Delta t} $$

We have already used this method to step our prognostic variables forward in time.

Solving the above for the future value of $C$ gives

$$ C^{n+1} = C^n + \Delta t \frac{d C}{dt} \bigg|^n $$

We apply our discretization of the diffusion operator to the current value of the field $C^n_j$, to get our formula for the future values:

$$ C_j^{n+1} = C_j^n + \frac{K \Delta t}{\Delta x^2} \left( C^n_{j+1} - 2 C^n_{j} + C^n_{j-1} \right)  $$

(except at the boundaries, where the diffusion operator is slightly different -- see above).

Together, this scheme is known as **Forward Time, Centered Space** or **FTCS**.

It is very simple to implement in `numpy` code.

In [ ]:
def step_forward(C, deltax, deltat, K=1):
    dudt = diffusion(C, deltax, K)
    return C + deltat * dudt

In [ ]:
K = 0.01
deltat = 0.125
deltat1 = deltat

C0 = gaussian(x, 0.5, 0.08)
C1 = step_forward(C0, deltax, deltat1, K)
fig, ax = plt.subplots(1)
ax.plot(x, C0, label='initial')
ax.plot(x, C1, label='next')
ax.legend()

Let's loop through a number of timesteps.

In [ ]:
#  regular resolution
J = 20
deltax = 1./J
xstag = np.linspace(0., 1., J+1)
x = xstag[:-1] + deltax/2

In [ ]:
C = gaussian(x, 0.5, 0.08)
niter = 11
for n in range(niter):
    C = step_forward(C, deltax, deltat1, K)
    plt.plot(x, C, label=n)
plt.legend()

The numerics were easy to implement, and the scheme seems to work very well! The results are physically sensible.

### Now, suppose that you wanted to **double** the spatial resolution

Try setting $J=40$ and repeat the above procedure.

What happens?

In [ ]:
#  double the resolution
scaling_factor = 2
J = J * scaling_factor
deltax = 1./J
xstag = np.linspace(0., 1., J+1)
x = xstag[:-1] + deltax/2

In [ ]:
C = gaussian(x, 0.5, 0.08)
for n in range(niter):
    C = step_forward(C, deltax, deltat1, K)
    plt.plot(x, C, label=n)
plt.legend()

Suddenly our scheme is producing numerical noise that grows in time and overwhelms to smooth physical solution we are trying to model.

**This is bad!**

What went wrong, and what can we do about it?

____________
<a id='section6'></a>

## 6. Numerical tests with a shorter timestep
____________

Going back to our Gaussian example, let's double the resolution but shorten the timestep by a factor of 4.


In [ ]:
#  double the resolution
# J = J1 * scaling_factor
deltax = 1./J
xstag = np.linspace(0., 1., J+1)
x = xstag[:-1] + deltax/2

In [ ]:
K = 0.01
#  The maximum stable timestep
deltat_max = deltax**2 / 2 / K
print( 'The maximum allowable timestep is %f' %deltat_max)

deltat = deltat1 / scaling_factor**2
print( '4x the previous timestep is %f' %deltat)

In [ ]:
C = gaussian(x, 0.5, 0.08)
for n in range(niter):
    for t in range(scaling_factor**2):
        C = step_forward(C, deltax, deltat, K)
    plt.plot(x, C, label=n)
plt.legend()

Success! The graph now looks like a smoother (higher resolution) version of our first integration with the coarser grid.

**But at a big cost**:  our calculation required 4 times more timesteps to do the same integration.

The total increase in computational cost was actally a factor of 8 to get a factor of 2 increase in spatial resolution.

____________
<a id='section7'></a>

## 7. The Matrix Format
____________


Collecting all the future time step on the lefthand side and the rest on the right, we have

$$ C_j^{n+1}  = K^* C^{n}_{j-1} + \left(1-2 K^*\right) C_j^n + K^* C^{n}_{j+1}  $$

in the interior, and at the boundaries:

$$ C_0^{n+1} = (1-K^*) C_0^{n} + K^* C_1^n $$

and

$$ C_{j-1}^{n+1} = K^* C_{j-2}^n + (1-K^*) C_{j-1}^n $$



### As a matrix problem

We can write this as a matrix equation

$$  \mathbf{C}^{n+1} = \mathbf{A} ~\mathbf{C}^n $$

where $\mathbf{C}$ is a $J\times1$ column vector giving the field $C(x)$ at a particular instant in time:

$$ \mathbf{C}^n = \left[ \begin{array}{c} 
C^n_0 \\
C^n_1  \\
C^n_2 \\
...  \\
C^n_{J-2} \\
C^n_{J-1} \\
\end{array}
\right] 
$$

and $\mathbf{C}^{n+1}$ is the same vector at $t^{n+1}$.

$\mathbf{A}$ is a $J\times J$ tridiagonal matrix:

$$ \mathbf{A} = \left[ \begin{array}{cccccccc}
 1-K^* & K^* & 0 & 0 & ... & 0 & 0 & 0 \\
 K^* & 1-2K^* & K^* & 0 & ... & 0 & 0 & 0 \\
 0 & K^* & 1-2K^* & K^* &... & 0 & 0 & 0 \\
 ... & ... & ... & ... & ... & ... & ... & ... \\
 0 & 0 & 0 & 0 & ... & K^* & 1-2K^* & K^* \\
 0 & 0 & 0 & 0 & ... & 0 & K^* & 1-K^* \\
\end{array}
\right] 
$$

# Coding in the matrix fashion

In [ ]:
import numpy as np
def A_diff(K, deltax, deltat, J):
    Ks = K*deltat/deltax**2
    A = np.diag(np.ones(J-1)*Ks, -1) + np.diag(np.ones(J)*(1-2*Ks), 0) + np.diag(np.ones(J-1)*Ks, 1)
    A[0,0]= 1-Ks  # BC1
    A[-1,-1] = 1- Ks # BC2
    return A
    

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

K =0.01
J = 20
deltax = 1/J
deltat= 0.0125
x=np.linspace(0,1,J)

C = np.zeros(J,).T
C[(x>0.4)&(x<0.6)]=1

A = A_diff(K, deltax, deltat, J)

for i in range(0,200):
    if i%30 == 0:
        plt.plot(x,C)
    C = np.matmul(A, C)
plt.show()

# Challenge:

1. Decrease the time step to 0.005, what happens?
2. Increase delta x, what will happen?
3. Increase the resolution to 100 grid points. What happen?
4. Change the number of grid pints to 20, now change the time step to 0.1. What happen?
5. Can you find the maximum grid resolution and time step?

# A more complex problem: advection-diffusion-source problem:
$$ \frac{\partial C}{\partial t} + U \frac{\partial C}{\partial x} = K \frac{\partial^2 C}{\partial x^2} + S(x) $$


The descretized format:

???

in the interior, and at the boundaries:

$$ \frac{C_0^{n+1} - C_0^n}{\Delta t} + \frac{U}{ \Delta x} \left( C^{n}_{1} - C^{n}_{0} \right) = \frac{K}{\Delta x^2} \left( C^{n}_1 - C^{n}_0 \right) + S_0^n$$

and

$$ \frac{C_{J-1}^{n+1} - C_{J-1}^n}{\Delta t} + \frac{U}{\Delta x} \left( C^{n}_{J-1} - C^{n}_{J-2} \right) = - \frac{K}{\Delta x^2} \left( C_{J-1}^{n} - C_{J-2}^{n} \right) + S_{J-1}^n$$


### As a matrix problem

We can write this as a matrix equation

$$  \mathbf{C}^{n+1} = \mathbf{A} ~\mathbf{C}^n + \mathbf{S}^{n}$$

where $\mathbf{C}$ is a $J\times1$ column vector giving the field $C(x)$ at a particular instant in time:

$$ \mathbf{C}^n = \left[ \begin{array}{c} 
C^n_0 \\
C^n_1  \\
C^n_2 \\
...  \\
C^n_{J-2} \\
C^n_{J-1} \\
\end{array}
\right] 
$$,

$$ \mathbf{S}^n = \left[ \begin{array}{c} 
S^n_0 \\
S^n_1  \\
S^n_2 \\
...  \\
S^n_{J-2} \\
S^n_{J-1} \\
\end{array}
\right] 
$$

and $\mathbf{C}^{n+1}$ is the same vector at $t^{n+1}$.

$\mathbf{A}$ is a $J\times J$ tridiagonal matrix:

$$ \mathbf{A} = \left[ \begin{array}{cccccccc}
 2 U^*-K^* & ? & 0 & 0 & ... & 0 & 0 & 0 \\
 ? & ? & ? & 0 & ... & 0 & 0 & 0 \\
 0 & ? & ? & ? &... & 0 & 0 & 0 \\
 ... & ... & ... & ... & ... & ... & ... & ... \\
 0 & 0 & 0 & 0 & ... & ? & ? & ? \\
 0 & 0 & 0 & 0 & ... & 0 & ? & 2 U^*-K^* \\
\end{array}
\right] 
$$

## Challenge:
Complete the codes below to realize the problem above

In [ ]:
import numpy as np
def A_diff_advection(K, deltax, deltat, J, U):
    Ks = # insert your answer here
    Us = # insert your answer here
    A = np.diag(np.ones(__)*(__), -1) + np.diag(np.ones(__)*(__), 0) + np.diag(np.ones(__)*(__), 1)
    A[0,0]= 2*Us-Ks  # BC1
    A[1,0]= Ks-2*Us
    A[-1,-2] = Ks-2*Us
    A[-1,-1] = 2*Us- Ks # BC2
    return A

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

K =0.01
J = 20
deltax = 1/J
deltat= 0.0125
x=np.linspace(0,1,J)
U=0.05
S=np.zeros_like(x)

C = np.zeros(J,).T
C[(x>0.2)&(x<0.4)]=1

A = A_diff_advection(K, deltax, deltat, J, U)

for i in range(0,200):
    if i%30 == 0:
        plt.plot(x,C)
    C = np.matmul(A, C) + S
plt.show()

# Challenge:

Rerun the code with the following initial conditions:

1. C[(x>0.3)&(x<0.4)]=1
2. C[x<0.4]=1 and C[x>=0.4]=0
3. C = np.arange(J)/J